Basic CNN

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable # 계산을 마친 후 기울기를 구할 수 있게 해주는 패키지
from six.moves import urllib

batch_size = 64

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)
#Load data
new_mirror = 'https://ossci-datasets.s3.amazonaws.com/mnist'
datasets.MNIST.resources = [
   ('/'.join([new_mirror, url.split('/')[-1]]), md5)
   for url, md5 in datasets.MNIST.resources
]

train_datasets = datasets.MNIST(root = './data/', train = True, transform = transforms.ToTensor(), download = True)
test_datasets = datasets.MNIST(root = './data/', train = False, transform = transforms.ToTensor(), download = True)

train_dataloader = torch.utils.data.DataLoader(dataset = train_datasets, batch_size = batch_size, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_datasets, batch_size = batch_size, shuffle = False)

#Define the Model
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    #28*28 -> conv1: (28-5)/1+1= 24 -> MaxPooling(2) : (24-2)/2+1 = 12 
    # -> conv2 : (12-5)/1+1 = 8 -> Maxpool : (8-2)/2+1 = 4   
    # 4*4*20 = 320
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
    self.mp = nn.MaxPool2d(2, 2)
    self.fc = nn.Linear(4*4*20, 10)
  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x))) 
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

model = Net()

optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_dataloader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)

    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 30 == 0:
      print('Train Epoch : {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(epoch, batch_idx*len(data), len(train_dataloader.dataset), 100.*batch_idx/len(train_dataloader), loss.item()))

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_dataloader:
    data, target = Variable(data, volatile = True), Variable(target)
    output = model(data)
    test_loss = F.nll_loss(output, target, size_average = False).data
    pred = output.data.max(1, keepdim = True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  test_loss /= len(test_dataloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                               len(test_dataloader.dataset),
                                                                               100. * correct / len(test_dataloader.dataset)))  
  
for epoch in range(1, 10):
  train(epoch)
  test()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch : 1 [0/60000 (0%)]	 Loss: 2.300617
Train Epoch : 1 [1920/60000 (3%)]	 Loss: 2.279958
Train Epoch : 1 [3840/60000 (6%)]	 Loss: 2.227157
Train Epoch : 1 [5760/60000 (10%)]	 Loss: 2.129974
Train Epoch : 1 [7680/60000 (13%)]	 Loss: 1.798168
Train Epoch : 1 [9600/60000 (16%)]	 Loss: 1.201898
Train Epoch : 1 [11520/60000 (19%)]	 Loss: 0.743099
Train Epoch : 1 [13440/60000 (22%)]	 Loss: 0.723994
Train Epoch : 1 [15360/60000 (26%)]	 Loss: 0.595065
Train Epoch : 1 [17280/60000 (29%)]	 Loss: 0.685671
Train Epoch : 1 [19200/60000 (32%)]	 Loss: 0.440995
Train Epoch : 1 [21120/60000 (35%)]	 Loss: 0.547690
Train Epoch : 1 [23040/60000 (38%)]	 Loss: 0.415244
Train Epoch : 1 [24960/60000 (42%)]	 Loss: 0.466438
Train Epoch : 1 [26880/60000 (45%)]	 Loss: 0.465986
Train Epoch : 1 [28800/60000 (48%)]	 Loss: 0.373933
Train Epoch : 1 [30720/60000 (51%)]	 Loss: 0.289974
Train Epoch : 1 [32640/60000 (54%)]	 Loss: 0.386763
Train Epoch : 1 [34560/60000 (58%)]	 Loss: 0.198527
Train Epoch : 1 [36480/6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0002, Accuracy: 9413/10000 (94%)

Train Epoch : 2 [0/60000 (0%)]	 Loss: 0.164978
Train Epoch : 2 [1920/60000 (3%)]	 Loss: 0.162050
Train Epoch : 2 [3840/60000 (6%)]	 Loss: 0.208544
Train Epoch : 2 [5760/60000 (10%)]	 Loss: 0.137271
Train Epoch : 2 [7680/60000 (13%)]	 Loss: 0.188263
Train Epoch : 2 [9600/60000 (16%)]	 Loss: 0.245003
Train Epoch : 2 [11520/60000 (19%)]	 Loss: 0.144125
Train Epoch : 2 [13440/60000 (22%)]	 Loss: 0.292327
Train Epoch : 2 [15360/60000 (26%)]	 Loss: 0.181908
Train Epoch : 2 [17280/60000 (29%)]	 Loss: 0.280806
Train Epoch : 2 [19200/60000 (32%)]	 Loss: 0.281063
Train Epoch : 2 [21120/60000 (35%)]	 Loss: 0.225415
Train Epoch : 2 [23040/60000 (38%)]	 Loss: 0.273177
Train Epoch : 2 [24960/60000 (42%)]	 Loss: 0.154823
Train Epoch : 2 [26880/60000 (45%)]	 Loss: 0.292257
Train Epoch : 2 [28800/60000 (48%)]	 Loss: 0.064496
Train Epoch : 2 [30720/60000 (51%)]	 Loss: 0.199154
Train Epoch : 2 [32640/60000 (54%)]	 Loss: 0.120483
Train Epoch : 2

CNN More layers

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable # 계산을 마친 후 기울기를 구할 수 있게 해주는 패키지
from six.moves import urllib

batch_size = 64

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)
#Load data
new_mirror = 'https://ossci-datasets.s3.amazonaws.com/mnist'
datasets.MNIST.resources = [
   ('/'.join([new_mirror, url.split('/')[-1]]), md5)
   for url, md5 in datasets.MNIST.resources
]

train_datasets = datasets.MNIST(root = './data/', train = True, transform = transforms.ToTensor(), download = True)
test_datasets = datasets.MNIST(root = './data/', train = False, transform = transforms.ToTensor(), download = True)

train_dataloader = torch.utils.data.DataLoader(dataset = train_datasets, batch_size = batch_size, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_datasets, batch_size = batch_size, shuffle = False)

#Define the Model
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 3, padding = 1)
    self.conv2 = nn.Conv2d(10, 20, kernel_size = 3, padding = 1)
    self.conv3 = nn.Conv2d(20, 40, kernel_size = 3,  stride = 2, padding = 0)
    self.mp = nn.MaxPool2d(2, 2)
    self.fc1 = nn.Linear(3*3*40, 180)
    self.fc2 = nn.Linear(180, 10)
  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x))) 
    x = F.relu(self.mp(self.conv2(x)))
    x = F.relu(self.conv3(x))
    x = x.view(in_size, -1)
    x = self.fc1(x)
    x = self.fc2(x)
    return F.log_softmax(x)

model = Net()

optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_dataloader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)

    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 30 == 0:
      print('Train Epoch : {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(epoch, batch_idx*len(data), len(train_dataloader.dataset), 100.*batch_idx/len(train_dataloader), loss.item()))

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_dataloader:
    data, target = Variable(data, volatile = True), Variable(target)
    output = model(data)
    test_loss = F.nll_loss(output, target, size_average = False).data
    pred = output.data.max(1, keepdim = True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  test_loss /= len(test_dataloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                               len(test_dataloader.dataset),
                                                                               100. * correct / len(test_dataloader.dataset)))  
  
for epoch in range(1, 10):
  train(epoch)
  test()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch : 1 [0/60000 (0%)]	 Loss: 2.292071
Train Epoch : 1 [1920/60000 (3%)]	 Loss: 2.304978
Train Epoch : 1 [3840/60000 (6%)]	 Loss: 2.292072
Train Epoch : 1 [5760/60000 (10%)]	 Loss: 2.297300
Train Epoch : 1 [7680/60000 (13%)]	 Loss: 2.292653
Train Epoch : 1 [9600/60000 (16%)]	 Loss: 2.291600
Train Epoch : 1 [11520/60000 (19%)]	 Loss: 2.299934
Train Epoch : 1 [13440/60000 (22%)]	 Loss: 2.266182
Train Epoch : 1 [15360/60000 (26%)]	 Loss: 2.260075
Train Epoch : 1 [17280/60000 (29%)]	 Loss: 2.257205
Train Epoch : 1 [19200/60000 (32%)]	 Loss: 2.211914
Train Epoch : 1 [21120/60000 (35%)]	 Loss: 2.195322
Train Epoch : 1 [23040/60000 (38%)]	 Loss: 1.972979
Train Epoch : 1 [24960/60000 (42%)]	 Loss: 1.870728
Train Epoch : 1 [26880/60000 (45%)]	 Loss: 1.457756
Train Epoch : 1 [28800/60000 (48%)]	 Loss: 1.163807
Train Epoch : 1 [30720/60000 (51%)]	 Loss: 0.784363
Train Epoch : 1 [32640/60000 (54%)]	 Loss: 0.941159
Train Epoch : 1 [34560/60000 (58%)]	 Loss: 0.447303
Train Epoch : 1 [36480/6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0004, Accuracy: 8854/10000 (89%)

Train Epoch : 2 [0/60000 (0%)]	 Loss: 0.412370
Train Epoch : 2 [1920/60000 (3%)]	 Loss: 0.334618
Train Epoch : 2 [3840/60000 (6%)]	 Loss: 0.543343
Train Epoch : 2 [5760/60000 (10%)]	 Loss: 0.198097
Train Epoch : 2 [7680/60000 (13%)]	 Loss: 0.303061
Train Epoch : 2 [9600/60000 (16%)]	 Loss: 0.278961
Train Epoch : 2 [11520/60000 (19%)]	 Loss: 0.300978
Train Epoch : 2 [13440/60000 (22%)]	 Loss: 0.165741
Train Epoch : 2 [15360/60000 (26%)]	 Loss: 0.248025
Train Epoch : 2 [17280/60000 (29%)]	 Loss: 0.407669
Train Epoch : 2 [19200/60000 (32%)]	 Loss: 0.342743
Train Epoch : 2 [21120/60000 (35%)]	 Loss: 0.291357
Train Epoch : 2 [23040/60000 (38%)]	 Loss: 0.468377
Train Epoch : 2 [24960/60000 (42%)]	 Loss: 0.309232
Train Epoch : 2 [26880/60000 (45%)]	 Loss: 0.151624
Train Epoch : 2 [28800/60000 (48%)]	 Loss: 0.324751
Train Epoch : 2 [30720/60000 (51%)]	 Loss: 0.337765
Train Epoch : 2 [32640/60000 (54%)]	 Loss: 0.258155
Train Epoch : 2

Inception Module

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable # 계산을 마친 후 기울기를 구할 수 있게 해주는 패키지
from six.moves import urllib

batch_size = 64

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)
#Load data
new_mirror = 'https://ossci-datasets.s3.amazonaws.com/mnist'
datasets.MNIST.resources = [
   ('/'.join([new_mirror, url.split('/')[-1]]), md5)
   for url, md5 in datasets.MNIST.resources
]

train_datasets = datasets.MNIST(root = './data/', train = True, transform = transforms.ToTensor(), download = True)
test_datasets = datasets.MNIST(root = './data/', train = False, transform = transforms.ToTensor(), download = True)

train_dataloader = torch.utils.data.DataLoader(dataset = train_datasets, batch_size = batch_size, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset = test_datasets, batch_size = batch_size, shuffle = False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#Define the Inception
class InceptionA(nn.Module):
  def __init__(self, in_channels):
    super(InceptionA, self).__init__()
    self.branch1 = nn.Conv2d(in_channels, 24, kernel_size = 1)
    
    self.branch2 = nn.Conv2d(in_channels, 16, kernel_size = 1)
    
    self.branch31 = nn.Conv2d(in_channels, 16, kernel_size = 1)
    self.branch32 = nn.Conv2d(16, 24, kernel_size = 5, padding = 2)

    self.branch41 = nn.Conv2d(in_channels, 16, kernel_size = 1)
    self.branch42 = nn.Conv2d(16, 24, kernel_size = 3, padding = 1)
    self.branch43 =nn.Conv2d(24, 24, kernel_size = 3, padding = 1)
  def forward(self, x):
    branch1 = F.avg_pool2d(x, kernel_size = 3, stride = 1, padding = 1)
    branch1 = self.branch1(branch1)

    branch2 = self.branch2(x)

    branch3 = self.branch31(x)
    branch3 = self.branch32(branch3)

    branch4 = self.branch41(x)
    branch4 = self.branch42(branch4)
    branch4 = self.branch43(branch4)

    output = [branch1, branch2, branch3, branch4]
    return torch.cat(output, 1)

    
#Define the Model
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
    self.conv2 = nn.Conv2d(88, 20, kernel_size = 5)

    self.inception1 = InceptionA(in_channels = 10)
    self.inception2 = InceptionA(in_channels = 20)

    self.mp = nn.MaxPool2d(2, 2)

    self.fc = nn.Linear(1408, 10)
  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x))) 
    x = self.inception1(x)
    x = F.relu(self.mp(self.conv2(x)))
    x = self.inception2(x)
    x = x.view(in_size, -1)
    x = self.fc(x)

    return F.log_softmax(x)

model = Net()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_dataloader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)

    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 30 == 0:
      print('Train Epoch : {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(epoch, batch_idx*len(data), len(train_dataloader.dataset), 100.*batch_idx/len(train_dataloader), loss.item()))

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_dataloader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    test_loss = F.nll_loss(output, target, size_average = False).data
    pred = output.data.max(1, keepdim = True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
  test_loss /= len(test_dataloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                                                                               len(test_dataloader.dataset),
                                                                               100. * correct / len(test_dataloader.dataset)))  
  
for epoch in range(1, 10):
  train(epoch)
  test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:81: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch : 1 [0/60000 (0%)]	 Loss: 2.301668
Train Epoch : 1 [1920/60000 (3%)]	 Loss: 2.294249
Train Epoch : 1 [3840/60000 (6%)]	 Loss: 2.275918
Train Epoch : 1 [5760/60000 (10%)]	 Loss: 2.284297
Train Epoch : 1 [7680/60000 (13%)]	 Loss: 2.228057
Train Epoch : 1 [9600/60000 (16%)]	 Loss: 2.095502
Train Epoch : 1 [11520/60000 (19%)]	 Loss: 1.369820
Train Epoch : 1 [13440/60000 (22%)]	 Loss: 0.866947
Train Epoch : 1 [15360/60000 (26%)]	 Loss: 0.552316
Train Epoch : 1 [17280/60000 (29%)]	 Loss: 0.689617
Train Epoch : 1 [19200/60000 (32%)]	 Loss: 0.453148
Train Epoch : 1 [21120/60000 (35%)]	 Loss: 0.491498
Train Epoch : 1 [23040/60000 (38%)]	 Loss: 0.559226
Train Epoch : 1 [24960/60000 (42%)]	 Loss: 0.560720
Train Epoch : 1 [26880/60000 (45%)]	 Loss: 0.487395
Train Epoch : 1 [28800/60000 (48%)]	 Loss: 0.309513
Train Epoch : 1 [30720/60000 (51%)]	 Loss: 0.319747
Train Epoch : 1 [32640/60000 (54%)]	 Loss: 0.211430
Train Epoch : 1 [34560/60000 (58%)]	 Loss: 0.246283
Train Epoch : 1 [36480/6

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0001, Accuracy: 9471/10000 (95%)

Train Epoch : 2 [0/60000 (0%)]	 Loss: 0.062704
Train Epoch : 2 [1920/60000 (3%)]	 Loss: 0.293146
Train Epoch : 2 [3840/60000 (6%)]	 Loss: 0.313729
Train Epoch : 2 [5760/60000 (10%)]	 Loss: 0.286529
Train Epoch : 2 [7680/60000 (13%)]	 Loss: 0.082134
Train Epoch : 2 [9600/60000 (16%)]	 Loss: 0.203096
Train Epoch : 2 [11520/60000 (19%)]	 Loss: 0.095195
Train Epoch : 2 [13440/60000 (22%)]	 Loss: 0.090687
Train Epoch : 2 [15360/60000 (26%)]	 Loss: 0.121489
Train Epoch : 2 [17280/60000 (29%)]	 Loss: 0.153419
Train Epoch : 2 [19200/60000 (32%)]	 Loss: 0.143193
Train Epoch : 2 [21120/60000 (35%)]	 Loss: 0.328656
Train Epoch : 2 [23040/60000 (38%)]	 Loss: 0.122188
Train Epoch : 2 [24960/60000 (42%)]	 Loss: 0.062477
Train Epoch : 2 [26880/60000 (45%)]	 Loss: 0.191015
Train Epoch : 2 [28800/60000 (48%)]	 Loss: 0.105018
Train Epoch : 2 [30720/60000 (51%)]	 Loss: 0.068528
Train Epoch : 2 [32640/60000 (54%)]	 Loss: 0.295231
Train Epoch : 2